In [125]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import numpy as np

In [147]:
def get_abs_income_data(filepath):

    year = Path(filepath).stem[0:4]
    year_str = str(year)

    df_raw = pd.read_csv(
        filepath,
        header=6,
        #names=["Postcode", "Fuel Type", "Vehicle Type", year_str],
        # dtype = {'Postcode': str, 'Fuel Type': str, 'Vehicle Type': str, year_str: int},
        # dtype=object,
        #index_col=False,
        #skipfooter=13,
        #engine="python",
        # nrows=1000,
    )
    df = df_raw.copy()
    return df

pi_df = get_abs_income_data("../data/raw/ABS/2021-personal-income-by-postcode.csv")
# rename first column to "Postcode"
pi_df = pi_df.rename(columns={pi_df.columns[0]: "Postcode"})
# filter out rows that don't start with 4 digits
pi_df = pi_df[pi_df["Postcode"].str.match(r"^\d{4}.*")]
# strip anything after the first 4 digits from the Postcode
pi_df["Postcode"] = pi_df["Postcode"].str.extract(r"(\d{4})")
# drop last two columns
#pi_df = pi_df.iloc[:, :-2]
# rename second column to $0-
pi_df = pi_df.rename(columns={pi_df.columns[1]: "$0-"})
total = pi_df["Total"]
ncols = len(pi_df.columns)
cols = pi_df.columns
data = pi_df[cols[1:ncols-2]]
# convert values in data to numeric type
data = data.apply(pd.to_numeric)
total = data.sum(axis=1)
# for each column in data, divide by total
frac = data.div(total, axis=0)
# income from header
weekly_income = data.columns.str.extract(r"\$(.*?)-")[0]
weekly_income[len(weekly_income)-1]='3,500' # yeah, hack I know - you figure the regex out then :)
# for all entries in income, replace commas with nothing
weekly_income = weekly_income.str.replace(',', '')
# convert income to number
weekly_income = pd.to_numeric(weekly_income)
func = lambda x: sum(np.asarray(x) * np.asarray(weekly_income))
mean_weekly_income = frac.apply(func, axis=1)
display(mean_weekly_income)
fig = px.histogram(mean_weekly_income)
fig.show()


1       1149.861211
2        929.844339
3       1004.928752
4       1403.205081
5       1525.037864
           ...     
2637     615.707373
2638     676.593673
2639     573.070064
2640     651.556858
2641    1132.073814
Length: 2641, dtype: float64